In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import re
import random
import pandas as pd

In [ ]:
# data = pd.read_csv('Textranked.csv')
# data.head()

# # data.dropna(inplace=True)
# # data.rename(columns={'본문':'question', '답변':'answer'}, inplace=True)
# # data.to_csv('prep_data_dotsplits.csv', index=False, encoding='utf-8')

In [ ]:
# SOS_token = 0
# EOS_token = 1

# class QNA:
#     def __init__(self, name):
#         self.name = name
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {0: "SOS", 1:"EOS"}
#         self.n_words = 2

#     def addSentence(self, sentence):
#         for word in sentence.split('.'):
#             self.addWord(word)

#     def addWord(self, word):
#         for sen in word:
#             if sen not in self.word2index:             
#                 self.word2index[sen] = self.n_words
#                 self.word2count[sen] = 1
#                 self.index2word[self.n_words] = sen
#                 self.n_words += 1
#             else:
#                 self.word2count[sen] += 1

In [ ]:
# 형태소 단위 딕셔너리 구축

SOS_token = 0
EOS_token = 1

!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

from konlpy.tag import Okt, Mecab
mecab = Mecab()

class QNA:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1:"EOS"}
        self.n_words = 2
        self.stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])

    def addSentence(self, sentence):
        for word in sentence.split('.'):
            self.addWord(word)

    def addWord(self, sen):
        for word in [ val for (val, nn) in mecab.pos(sen) if (val, nn) not in self.stopword and nn in ('NNG', 'NNP', 'VV', 'VA')]:
            if word not in self.word2index:             
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word[self.n_words] = word
                self.n_words += 1
            else:
              self.word2count[word] += 1

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
def normalizeString(s):
    s = re.sub(r"([.!?])", r" ", s)
    s = re.sub(r"[^가-힣.!?]+", r" ", s)
    return s

In [ ]:
def readchat(question, answer, reverse=False):
    print("Reading lines...")

    lines = open('/content/drive/MyDrive/Textranked.csv', encoding='utf-8').read().strip().split('\n')[1:]
    
    pairs = [[normalizeString(s) for s in l.split(',')] for l in lines]

    if reverse:
        paris = [list(reversed(p)) for p in pairs]
        input_q = QNA(answer)
        output_a = QNA(question)
    else:
        input_q = QNA(question)
        output_a = QNA(answer)
    
    return input_q, output_a, pairs

In [ ]:
MAX_LENGTH = 2000
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(question, answer, reverse=False):
    input_q, output_a, pairs = readchat(question, answer, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    for pair in pairs:
        input_q.addSentence(pair[0])
        output_a.addSentence(pair[1])
    print("Counted words:")
    print(input_q.name, input_q.n_words)
    print(output_a.name, output_a.n_words)
    return input_q, output_a, pairs

input_q, output_a, pairs = prepareData('question', 'answer', True)
print(random.choice(pairs))

Reading lines...
Read 18165 sentence pairs
Counted words:
answer 12518
question 7755
['년 전 임금 퇴직금 체불임금이 약 만 원으로 노동청 신고 법률구조공단 신청 사업주에게 공증을 받아 집행문으로 강제집행 민사소송으로 처벌을 원한다고 하였으나 카드사 배당 하루 전날 회생 지급 중지 명령이 떨어졌고 그 이후 회생 개시되어 체당금 여만 원을 받고 잊고 살던 중 어떤 이유에서 회생이 폐지되었어요', '회생이 폐지되었다면 남은 체불임금을 추심할 수 있으며 이전에 받은 강제집행권원 및 회생 채권자 목록을 통하여 사업주 및 회사의 재산에 압류를 신청하셔야 할 것으로 보입니다 어렵게 생각하시지 마시고 법률사무소에서 방문 및 전화상담을 받아보시기 바랍니다']


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [ ]:
def indexesFromSentence(QNA, sentence):
    word2index_list = []
    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
    for sen in sentence.split('.'):
        for word in [ val for (val, nn) in mecab.pos(sen) if (val, nn) not in stopword and nn in ('NNG', 'NNP', 'VV', 'VA')]:
            word2index_list.append(QNA.word2index[word])
    return word2index_list


def tensorFromSentence(QNA, sentence):
    indexes = indexesFromSentence(QNA, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=DEVICE).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_q, pair[0])
    target_tensor = tensorFromSentence(output_a, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    if input_length >= max_length :
        input_length = max_length - 1

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=DEVICE)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=DEVICE)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing 포함: 목표를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_q, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=DEVICE)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=DEVICE)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_a.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_q.n_words, hidden_size).to(DEVICE)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_a.n_words, dropout_p=0.1).to(DEVICE)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

6m 9s (- 86m 15s) (5000 6%) 5.1307
10m 59s (- 71m 24s) (10000 13%) 4.8259
15m 50s (- 63m 20s) (15000 20%) 4.6368
20m 43s (- 56m 58s) (20000 26%) 4.5942
25m 27s (- 50m 54s) (25000 33%) 4.5476
30m 11s (- 45m 17s) (30000 40%) 4.5033
35m 4s (- 40m 4s) (35000 46%) 4.5067
39m 57s (- 34m 57s) (40000 53%) 4.5902
44m 44s (- 29m 49s) (45000 60%) 4.4969
49m 23s (- 24m 41s) (50000 66%) 4.4146
54m 14s (- 19m 43s) (55000 73%) 4.3679
59m 16s (- 14m 49s) (60000 80%) 4.5724
64m 12s (- 9m 52s) (65000 86%) 4.4836
68m 57s (- 4m 55s) (70000 93%) 4.4139
73m 46s (- 0m 0s) (75000 100%) 4.4077


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
PATH = './Seq2Seq_mecab_encoder1.pth'
torch.save(encoder1.state_dict(), PATH)

In [ ]:
PATH = './Seq2Seq_mecab_encoder1_model.pth'
torch.save(encoder1, PATH)

In [ ]:
PATH = './Seq2Seq_mecab_attn_decoder1.pth'
torch.save(attn_decoder1.state_dict(), PATH)

In [ ]:
PATH = './Seq2Seq_mecab_attn_decoder1_model.pth'
torch.save(attn_decoder1.state_dict(), PATH)

In [ ]:
PATH = '/Seq2Seq_mecab_attn_decoder1.pth'
attn_decoder1.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
sentence2 = '직원과 사장으로 만나신분에게 대가성없이 통장을 대여해드렸습니다. 제 명의로 사업자를 내서 사업을 했으나 빛뿐이였습니다.'

In [ ]:
sentence1 = pairs[0][0]
sentence1

'정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다 아버지가 제 명의로 사업자를 내서 사업을 했으나 늘어나는 것은 빛뿐이었습니다 더 이상은 방관할 수가 없어 아버지에게 제 명의로 해드린 것을 모두 정지시키려고 하는 상황입니다 어머니 명의로 된 집 아버지 어머니 함께 거주를 매도하려고 하는데 그 과정에서 아버지가 방해를 하고 가구 이동을 못 시키게 하고 그러면 어떤 방법을 취할 수 있을까요 두 번째 질문입니다 제 명의로 된 차량을 아버지가 사용하고 계셨는데 제가 도로 갖고 와도 될까요'

In [ ]:
input_q.word2index

{'상황': 2,
 '힘들': 3,
 '질문': 4,
 '아버지': 5,
 '명의': 6,
 '사업자': 7,
 '내': 8,
 '사업': 9,
 '늘어나': 10,
 '빛': 11,
 '이상': 12,
 '방관': 13,
 '없': 14,
 '정지': 15,
 '어머니': 16,
 '집': 17,
 '거주': 18,
 '매도': 19,
 '과정': 20,
 '방해': 21,
 '이동': 22,
 '시키': 23,
 '그러': 24,
 '방법': 25,
 '차량': 26,
 '사용': 27,
 '갖': 28,
 '년': 29,
 '전': 30,
 '대포': 31,
 '통장이': 32,
 '피해': 33,
 '재판': 34,
 '결과': 35,
 '불기소': 36,
 '유통': 37,
 '지': 38,
 '받': 39,
 '상대편': 40,
 '부당이득': 41,
 '금': 42,
 '기소': 43,
 '걱정': 44,
 '이유': 45,
 '통지': 46,
 '서류': 47,
 '있': 48,
 '계좌': 49,
 '입금': 50,
 '부분': 51,
 '형': 52,
 '공인': 53,
 '증서': 54,
 '가지': 55,
 '주식': 56,
 '투자': 57,
 '자금': 58,
 '손실': 59,
 '지인': 60,
 '누나': 61,
 '그때': 62,
 '말': 63,
 '정도': 64,
 '최소한': 65,
 '남자': 66,
 '친구': 67,
 '돈': 68,
 '만들': 69,
 '카카오톡': 70,
 '메시지': 71,
 '도의': 72,
 '책임': 73,
 '주': 74,
 '조건': 75,
 '나쁘': 76,
 '천': 77,
 '단독': 78,
 '소행': 79,
 '가능': 80,
 '거짓말': 81,
 '기존': 82,
 '얘기': 83,
 '다르': 84,
 '일': 85,
 '진행': 86,
 '고소': 87,
 '여쭙': 88,
 '때': 89,
 '벌': 90,
 '좋': 91,
 '기억': 92,
 '상태': 93,
 '

In [ ]:
evaluate(encoder1, attn_decoder1, sentence1)

(['결론', '말씀', '드리', '경우', '<EOS>'],
 tensor([[0.0005, 0.0006, 0.0006,  ..., 0.0002, 0.0009, 0.0003],
         [0.0005, 0.0004, 0.0008,  ..., 0.0004, 0.0005, 0.0005],
         [0.0004, 0.0004, 0.0007,  ..., 0.0005, 0.0003, 0.0002],
         [0.0007, 0.0007, 0.0009,  ..., 0.0004, 0.0006, 0.0005],
         [0.0003, 0.0005, 0.0009,  ..., 0.0004, 0.0004, 0.0007]]))

In [ ]:
evaluate(encoder1, attn_decoder1, sentence2)

(['결론', '말씀', '드리', '회사', '회사', '회사', '회사', '취소', '<EOS>'],
 tensor([[0.0003, 0.0004, 0.0006,  ..., 0.0005, 0.0011, 0.0003],
         [0.0006, 0.0004, 0.0008,  ..., 0.0005, 0.0003, 0.0005],
         [0.0005, 0.0003, 0.0005,  ..., 0.0006, 0.0003, 0.0002],
         ...,
         [0.0006, 0.0009, 0.0002,  ..., 0.0005, 0.0008, 0.0002],
         [0.0006, 0.0011, 0.0002,  ..., 0.0005, 0.0008, 0.0002],
         [0.0002, 0.0004, 0.0003,  ..., 0.0006, 0.0006, 0.0005]]))

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

> 어떻게된겨져 그냥 문자랑 상관없이 같은 피해자분들처럼 신청서 작성해서 법원에 송부하면 되나요
= 이성권 변호사 사시 회입니다 형사변호사 대전 법원 앞 형사 전문 이성권 법률사무소 성범죄 민사 상담 성공사례 대전형사이혼전문변호사법률사무소변호사 법률사무소 가연의 이성권 변호사 사시 회입니다
< 결론 말씀 드리 필요 집행 권원 강제집행 방법 아래 내용 참고 채권 추심 절차 시작 부동산 강제집행 채권 추심 효과 방법 부동산 강제집행 경매 신청 경매 비용 필요 경매 비용 경락 대금 최우선 회수 경매 시작 의 경우 채무자 경매 취하 요청 자발 채무 변제 금채 압류 채권 추심 쉽 방법 채무자 예금 추심 채권 특정 사전 조사 필요 재산 명시 신청 재산 조회 채권자 법원 채무자 강제집행 대상 재산 재산 일정 기간 처분 상황 명시 재산 목록 제출 명령 재산 명시 신청 <EOS>

> 최근 전자소송으로 민사 소액 진행 중에 있습니다 경찰서에서 조사를 다 받고 형사소송까지 걸어놓은 상태입니다 오늘 변론 기일 통지서가 날라와서 월에 법원에 출석하라고 합니다 그런데 피고는 현재 빚 갚을 능력이 안되어 구속되어 있는 상태로 알고 있는데 반론을 제기했다는 건가요 그래서 월에 출석하라는 건지 궁금합니다 재판은 또 어떻게 진행될까요
= 상대방이 답변서를 제출하였다면 변론 기일이 잡힙니다
< 결론 말씀 드리 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 피고 원고 피고 원고 피고 원고 피고 원고 피고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원고 피고 원

In [ ]:
pairs